import timm
for model in timm.list_models():
  print(model)

In [ ]:
!pip install fvcore
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=130b42bc0c2c0d268771609ec084258c327af6c6fde32ec8a0cfcb00d4a01b84
  Stored in directory: /root/.cache/pip/wheels/ed/9f/a5/e4f5b27454ccd4596bd8b62432c7d6b1ca9fa22aef9d70a16a
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=a61053fce864af48bfa8cf1a5108e46f05eb5250e0776b13f6749ce346c9cfec
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built fvcore iopath


In [ ]:

%cd /content/
!git clone https://github.com/lht1321-netizen/ml_project.git
%cd ml_project
!git checkout 3887cb3acd60c8e6731d7b44bc15344607adabe9

%cd /content/
!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2

import sys
sys.path.append('/content/ml_project/Mask2Former')
sys.path.append('/content/detectron2')

/content
Cloning into 'ml_project'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 418 (delta 85), reused 6 (delta 6), pack-reused 199 (from 1)
Receiving objects: 100% (418/418), 11.17 MiB | 9.10 MiB/s, done.
Resolving deltas: 100% (156/156), done.
/content/ml_project
Note: switching to '3887cb3acd60c8e6731d7b44bc15344607adabe9'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 3887cb3 Add files via upload

In [ ]:
import math
import numpy as np
from typing import List

import torch.optim as optim
import torch
from torch import nn,Tensor
import torch.nn.functional as F
import torch.nn.init as init
import torchvision
import torch.distributed as tdist

from collections import OrderedDict
from scipy.optimize import linear_sum_assignment
from torchvision.ops.boxes import box_area
import timm

from detectron2.modeling import ShapeSpec
from ml_project.Mask2Former.pixel_decoder.pixel_decoder_v2 import MSDeform_attention_pixel_decoder

/content/ml_project/Mask2Former/pixel_decoder/ops/modules/ms_deform_attn.py:94: SyntaxWarning: invalid escape sequence '\s'
  :param input_flatten               (N, \sum_{l=0}^{L-1} H_l \cdot W_l, C)


EFDTRModel

In [ ]:
"""D2SwinTransformer"""

class D2SwinTransformer(nn.Module):
  def __init__(self,model_name='swin_base_patch4_window7_224'):
    super().__init__()
    self.backbone=timm.create_model(model_name,
                                    pretrained=True,
                                    features_only=True,
                                    out_indices=(0,1,2,3))
    self._out_features = ["res2", "res3", "res4", "res5"]
    self._out_feature_strides = {"res2": 4, "res3": 8, "res4": 16, "res5": 32}
    self._out_feature_channels = {
        "res2": self.backbone.feature_info[0]["num_chs"],
        "res3": self.backbone.feature_info[1]["num_chs"],
        "res4": self.backbone.feature_info[2]["num_chs"],
        "res5": self.backbone.feature_info[3]["num_chs"]}

  def forward(self,x):
    features=self.backbone(x)
    features=[f.permute(0,3,1,2).contiguous() for f in features]
    return{
        "res2":features[0],
        "res3":features[1],
        "res4":features[2],
        "res5":features[3]
    }

  def output_shape(self):
    return {
        name: ShapeSpec(channels=self._out_feature_channels[name], stride=self._out_feature_strides[name])
        for name in self._out_features
    }

# @title
def get_activation(act: str, inpace: bool=True):
    act = act.lower()
    if act == 'silu':
        m = nn.SiLU()
    elif act == 'relu':
        m = nn.ReLU()
    elif act == 'leaky_relu':
        m = nn.LeakyReLU()
    elif act == 'silu':
        m = nn.SiLU()
    elif act == 'gelu':
        m = nn.GELU()
    elif act is None:
        m = nn.Identity()
    elif isinstance(act, nn.Module):
        m = act
    else:
        raise RuntimeError('')
    if hasattr(m, 'inplace'):
        m.inplace = inpace
    return m


def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=-1)

def box_xyxy_to_cxcywh(x):
    x0, y0, x1, y1 = x.unbind(-1)
    b = [(x0 + x1) / 2, (y0 + y1) / 2,
         (x1 - x0), (y1 - y0)]
    return torch.stack(b, dim=-1)

def inverse_sigmoid(x: torch.Tensor, eps: float=1e-5) -> torch.Tensor:
    x = x.clip(min=0., max=1.)
    return torch.log(x.clip(min=eps) / (1 - x).clip(min=eps))

def get_contrastive_denoising_training_group(targets,
                                             num_classes,
                                             num_queries,
                                             class_embed,
                                             num_denoising=100,
                                             label_noise_ratio=0.5,
                                             box_noise_scale=1.0,):
    """cnd"""
    if num_denoising <= 0:
        return None, None, None, None

    num_gts = [len(t['labels']) for t in targets]
    device = targets[0]['labels'].device

    max_gt_num = max(num_gts)
    if max_gt_num == 0:
        return None, None, None, None

    num_group = num_denoising // max_gt_num
    num_group = 1 if num_group == 0 else num_group
    # pad gt to max_num of a batch
    bs = len(num_gts)

    input_query_class = torch.full([bs, max_gt_num], num_classes, dtype=torch.int32, device=device)
    input_query_bbox = torch.zeros([bs, max_gt_num, 4], device=device)
    pad_gt_mask = torch.zeros([bs, max_gt_num], dtype=torch.bool, device=device)

    for i in range(bs):
        num_gt = num_gts[i]
        if num_gt > 0:
            input_query_class[i, :num_gt] = targets[i]['labels']
            input_query_bbox[i, :num_gt] = targets[i]['boxes']
            pad_gt_mask[i, :num_gt] = 1
    # each group has positive and negative queries.
    input_query_class = input_query_class.tile([1, 2 * num_group])
    input_query_bbox = input_query_bbox.tile([1, 2 * num_group, 1])
    pad_gt_mask = pad_gt_mask.tile([1, 2 * num_group])
    # positive and negative mask
    negative_gt_mask = torch.zeros([bs, max_gt_num * 2, 1], device=device)
    negative_gt_mask[:, max_gt_num:] = 1
    negative_gt_mask = negative_gt_mask.tile([1, num_group, 1])
    positive_gt_mask = 1 - negative_gt_mask
    # contrastive denoising training positive index
    positive_gt_mask = positive_gt_mask.squeeze(-1) * pad_gt_mask
    dn_positive_idx = torch.nonzero(positive_gt_mask)[:, 1]
    dn_positive_idx = torch.split(dn_positive_idx, [n * num_group for n in num_gts])
    # total denoising queries
    num_denoising = int(max_gt_num * 2 * num_group)

    if label_noise_ratio > 0:
        mask = torch.rand_like(input_query_class, dtype=torch.float) < (label_noise_ratio * 0.5)
        # randomly put a new one here
        new_label = torch.randint_like(mask, 0, num_classes, dtype=input_query_class.dtype)
        input_query_class = torch.where(mask & pad_gt_mask, new_label, input_query_class)

    # if label_noise_ratio > 0:
    #     input_query_class = input_query_class.flatten()
    #     pad_gt_mask = pad_gt_mask.flatten()
    #     # half of bbox prob
    #     # mask = torch.rand(input_query_class.shape, device=device) < (label_noise_ratio * 0.5)
    #     mask = torch.rand_like(input_query_class) < (label_noise_ratio * 0.5)
    #     chosen_idx = torch.nonzero(mask * pad_gt_mask).squeeze(-1)
    #     # randomly put a new one here
    #     new_label = torch.randint_like(chosen_idx, 0, num_classes, dtype=input_query_class.dtype)
    #     # input_query_class.scatter_(dim=0, index=chosen_idx, value=new_label)
    #     input_query_class[chosen_idx] = new_label
    #     input_query_class = input_query_class.reshape(bs, num_denoising)
    #     pad_gt_mask = pad_gt_mask.reshape(bs, num_denoising)

    if box_noise_scale > 0:
        known_bbox = box_cxcywh_to_xyxy(input_query_bbox)
        diff = torch.tile(input_query_bbox[..., 2:] * 0.5, [1, 1, 2]) * box_noise_scale
        rand_sign = torch.randint_like(input_query_bbox, 0, 2) * 2.0 - 1.0
        rand_part = torch.rand_like(input_query_bbox)
        rand_part = (rand_part + 1.0) * negative_gt_mask + rand_part * (1 - negative_gt_mask)
        rand_part *= rand_sign
        known_bbox += rand_part * diff
        known_bbox.clip_(min=0.0, max=1.0)
        input_query_bbox = box_xyxy_to_cxcywh(known_bbox)
        input_query_bbox = inverse_sigmoid(input_query_bbox)

    # class_embed = torch.concat([class_embed, torch.zeros([1, class_embed.shape[-1]], device=device)])
    # input_query_class = torch.gather(
    #     class_embed, input_query_class.flatten(),
    #     axis=0).reshape(bs, num_denoising, -1)
    # input_query_class = class_embed(input_query_class.flatten()).reshape(bs, num_denoising, -1)
    input_query_class = class_embed(input_query_class)

    tgt_size = num_denoising + num_queries
    # attn_mask = torch.ones([tgt_size, tgt_size], device=device) < 0
    attn_mask = torch.full([tgt_size, tgt_size], False, dtype=torch.bool, device=device)
    # match query cannot see the reconstruction
    attn_mask[num_denoising:, :num_denoising] = True

    # reconstruct cannot see each other
    for i in range(num_group):
        if i == 0:
            attn_mask[max_gt_num * 2 * i: max_gt_num * 2 * (i + 1), max_gt_num * 2 * (i + 1): num_denoising] = True
        if i == num_group - 1:
            attn_mask[max_gt_num * 2 * i: max_gt_num * 2 * (i + 1), :max_gt_num * i * 2] = True
        else:
            attn_mask[max_gt_num * 2 * i: max_gt_num * 2 * (i + 1), max_gt_num * 2 * (i + 1): num_denoising] = True
            attn_mask[max_gt_num * 2 * i: max_gt_num * 2 * (i + 1), :max_gt_num * 2 * i] = True

    dn_meta = {
        "dn_positive_idx": dn_positive_idx,
        "dn_num_group": num_group,
        "dn_num_split": [num_denoising, num_queries]
    }

    # print(input_query_class.shape) # torch.Size([4, 196, 256])
    # print(input_query_bbox.shape) # torch.Size([4, 196, 4])
    # print(attn_mask.shape) # torch.Size([496, 496])

    return input_query_class, input_query_bbox, attn_mask, dn_meta

"""MLP"""

# @title
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, act='relu'):
        super().__init__()
        self.num_layers = num_layers
        h = [hidden_dim] * (num_layers - 1)
        self.layers = nn.ModuleList(nn.Linear(n, k) for n, k in zip([input_dim] + h, h + [output_dim]))
        self.act = nn.Identity() if act is None else get_activation(act)

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = self.act(layer(x)) if i < self.num_layers - 1 else layer(x)
        return x

"""_get_encoder_input"""

# @title
class _get_encoder_input(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, feats):
      feat_flatten = []
      spatial_shapes = []
      level_start_index = [0, ]
      for i, feat in enumerate(feats):
          _, _, h, w = feat.shape
          feat_flatten.append(feat.flatten(2).permute(0, 2, 1))
          spatial_shapes.append([h, w])
          level_start_index.append(h * w + level_start_index[-1])

      feat_flatten = torch.cat(feat_flatten, 1)
      level_start_index.pop()
      return (feat_flatten, spatial_shapes, level_start_index)

"""_generate_anchors"""

# @title
class _generate_anchors(nn.Module):
  def __init__(self,
                eps=1e-2,
                feat_strides=[4, 8, 16, 32],
                eval_spatial_size=[224, 224],
                ):
    super().__init__()
    self.eps = eps
    self.feat_strides = feat_strides
    self.eval_spatial_size = eval_spatial_size
  def forward(self,
              spatial_shapes=None,
              grid_size=0.05,
              dtype=torch.float32,
              device='cpu'):
      if spatial_shapes is None:
          spatial_shapes = [[int(self.eval_spatial_size[0] / s), int(self.eval_spatial_size[1] / s)]
              for s in self.feat_strides
          ]
      anchors = []
      for lvl, (h, w) in enumerate(spatial_shapes):
          grid_y, grid_x = torch.meshgrid(\
              torch.arange(end=h, dtype=dtype), \
              torch.arange(end=w, dtype=dtype), indexing='ij')
          grid_xy = torch.stack([grid_x, grid_y], -1)
          valid_WH = torch.tensor([w, h]).to(dtype)
          grid_xy = (grid_xy.unsqueeze(0) + 0.5) / valid_WH
          wh = torch.ones_like(grid_xy) * grid_size * (2.0 ** lvl)
          anchors.append(torch.concat([grid_xy, wh], -1).reshape(-1, h * w, 4))

      anchors = torch.concat(anchors, 1).to(device)
      valid_mask = ((anchors > self.eps) * (anchors < 1 - self.eps)).all(-1, keepdim=True)
      anchors = torch.log(anchors / (1 - anchors))
      # anchors = torch.where(valid_mask, anchors, float('inf'))
      # anchors[valid_mask] = torch.inf # valid_mask [1, 8400, 1]
      anchors = torch.where(valid_mask, anchors, torch.inf)

      return anchors, valid_mask

"""_get_decoder_input"""

# @title
class _get_decoder_input(nn.Module):
  def __init__(self,
               hidden_dim=256,
               num_queries=300,
               num_classes=80,
               learnt_init_query=False,
               ):

    super().__init__()

    self.hidden_dim = hidden_dim
    self.num_classes = num_classes
    self.num_queries = num_queries

    self.enc_output = nn.Sequential(
        nn.Linear(hidden_dim, hidden_dim),
        nn.LayerNorm(hidden_dim,)
    )

    self.enc_score_head = nn.Linear(hidden_dim, num_classes)
    self.enc_bbox_head = MLP(hidden_dim, hidden_dim, 4, num_layers=3)

    self.learnt_init_query = learnt_init_query
    if learnt_init_query:
      self.tgt_embed = nn.Embedding(num_queries, hidden_dim)

  def forward(self,
              memory,
              anchors,
              valid_mask,
              denoising_class=None,
              denoising_bbox_unact=None):
      bs, _, _ = memory.shape

      # memory = torch.where(valid_mask, memory, 0)
      memory = valid_mask.to(memory.dtype) * memory  # TODO fix type error for onnx export

      output_memory = self.enc_output(memory)

      enc_outputs_class = self.enc_score_head(output_memory)
      enc_outputs_coord_unact = self.enc_bbox_head(output_memory) + anchors

      _, topk_ind = torch.topk(enc_outputs_class.max(-1).values, self.num_queries, dim=1)

      reference_points_unact = enc_outputs_coord_unact.gather(dim=1, \
          index=topk_ind.unsqueeze(-1).repeat(1, 1, enc_outputs_coord_unact.shape[-1]))

      enc_topk_bboxes = F.sigmoid(reference_points_unact)
      if denoising_bbox_unact is not None:
          reference_points_unact = torch.concat(
              [denoising_bbox_unact, reference_points_unact], 1)

      enc_topk_logits = enc_outputs_class.gather(dim=1, \
          index=topk_ind.unsqueeze(-1).repeat(1, 1, enc_outputs_class.shape[-1]))

      # extract region features
      if self.learnt_init_query:
          target = self.tgt_embed.weight.unsqueeze(0).tile([bs, 1, 1])
      else:
          target = output_memory.gather(dim=1, \
              index=topk_ind.unsqueeze(-1).repeat(1, 1, output_memory.shape[-1]))
          target = target.detach()

      if denoising_class is not None:
          target = torch.concat([denoising_class, target], 1)

      return target, reference_points_unact.detach(), enc_topk_bboxes, enc_topk_logits

"""EFDEncoder"""

# @title
class EFDEncoder(nn.Module):

    def __init__(
        self,
        in_dim: int = 256,      # query feature 차원 (C)
        hidden_dim: int = 256,  # EFD 토큰의 임베딩 차원 (D)
        num_harmonics: int = 1  # n (order) → L = 4n + 1
    ):
        super().__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.num_harmonics = num_harmonics

        # EFD 계수 개수 (논문 figure: Length = 4n + 1)
        self.num_coeffs = 4 * num_harmonics + 1  # cx, cy, A1,B1,D1,...,Dn

        # 1) query feature → EFD 계수 벡터 [B,Q,L]
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, self.num_coeffs)
        )

        # 2) scalar coefficient → vector token 변환 (공유 projection)
        self.scalar_to_vec = nn.Linear(1, hidden_dim)

        # 3) 각 coefficient slot(0~L-1)에 대한 위치 임베딩
        self.slot_pos_embed = nn.Embedding(self.num_coeffs, hidden_dim)

    def forward(self, query_feats: torch.Tensor):
        """
        query_feats: [B, Q, C]

        return:
            efd_coeffs: [B, Q, L]
            efd_tokens: [B, Q, L, hidden_dim]
        """
        B, Q, C = query_feats.shape
        device = query_feats.device

        # (1) EFD 계수 예측: [B,Q,C] → [B,Q,L]
        efd_coeffs = self.mlp(query_feats)  # [B, Q, L]

        # (2) scalar → vector 변환
        #     각 scalar coeff를 hidden_dim짜리 벡터로 만들고,
        #     slot별 positional embedding을 더해 EFD Decoder용 토큰 생성
        L = self.num_coeffs

        coeffs_flat = efd_coeffs.view(B * Q * L, 1)           # [B*Q*L, 1]
        vecs_flat = self.scalar_to_vec(coeffs_flat)           # [B*Q*L, D]
        vecs = vecs_flat.view(B, Q, L, self.hidden_dim)       # [B,Q,L,D]

        slot_ids = torch.arange(L, device=device)             # [L]
        slot_pos = self.slot_pos_embed(slot_ids)              # [L,D]
        slot_pos = slot_pos.view(1, 1, L, self.hidden_dim)    # [1,1,L,D]

        efd_tokens = vecs + slot_pos                          # [B,Q,L,D]

        return efd_coeffs, efd_tokens

"""self attention + add & norm"""

# @title

class self_attention(nn.Module):
    #c_dim: feature dimension = query dimension
    #n_head: 멀티해더 수
    #논문에서 dropout이 0일 때가 퍼포먼스가 가장 좋았다고 해서 default를 0으로 설정
    def __init__(self, C_dim, n_head, dropout=0.0):
        super(self_attention, self).__init__()
        self.self_attn = nn.MultiheadAttention(C_dim, n_head, dropout=dropout, batch_first=True)

        self.dropout = nn.Dropout(dropout)

        self.norm = nn.LayerNorm(C_dim)

        self.reset_parameters()

    #parameter 초기화, xavier_uniform_ 사용
    def reset_parameters(self):
        for par in self.parameters():
            if par.dim() > 1:
                nn.init.xavier_uniform_(par)

    # query와 key에 위치 임베딩 더하기 위해 구현
    def add_pos_embedding(self, x: Tensor, pos: Tensor) -> Tensor:
        return x if pos is None else x + pos

    # x_last: 그전 단계의 출력값
    # mask: 마스크 텐서 / 키 마스크는 없는 것으로 설정(input으로 넣기 전에 전처리 따로 할 예정)
    # query_pos: 쿼리 포지션 임베딩 (쿼리가 어떤 위치를 나타내는지 알려주는 임베딩)
    # self attention이기 때문에 key = value
    def forward(self, x_last, mask = None, query_pos =None):
        # self attention
        Query_and_Key = self.add_pos_embedding(x_last, query_pos)

        self_attened = self.self_attn(
            query = Query_and_Key,
            key = Query_and_Key,
            value = x_last,
            attn_mask = mask,
            key_padding_mask = None
            )[0]

        # add & norm
        # 논문에 나오는 add&norm 부분을 self attention 모듈 안에 구현
        # add: x + f(x) / norm : 정규화(LayerNorm)
        x_next = x_last + self.dropout(self_attened)
        x_next = self.norm(x_next)

        return x_next

"""deformable_attention_core_func"""

# @title
def deformable_attention_core_func(value, value_spatial_shapes, sampling_locations, attention_weights):
    """
    Args:
        value (Tensor): [bs, value_length, n_head, c]
        value_spatial_shapes (Tensor|List): [n_levels, 2]
        value_level_start_index (Tensor|List): [n_levels]
        sampling_locations (Tensor): [bs, query_length, n_head, n_levels, n_points, 2]
        attention_weights (Tensor): [bs, query_length, n_head, n_levels, n_points]

    Returns:
        output (Tensor): [bs, Length_{query}, C]
    """
    bs, _, n_head, c = value.shape
    _, Len_q, _, n_levels, n_points, _ = sampling_locations.shape

    split_shape = [h * w for h, w in value_spatial_shapes]
    value_list = value.split(split_shape, dim=1)
    sampling_grids = 2 * sampling_locations - 1
    sampling_value_list = []
    for level, (h, w) in enumerate(value_spatial_shapes):
        # N_, H_*W_, M_, D_ -> N_, H_*W_, M_*D_ -> N_, M_*D_, H_*W_ -> N_*M_, D_, H_, W_
        value_l_ = value_list[level].flatten(2).permute(
            0, 2, 1).reshape(bs * n_head, c, h, w)
        # N_, Lq_, M_, P_, 2 -> N_, M_, Lq_, P_, 2 -> N_*M_, Lq_, P_, 2
        sampling_grid_l_ = sampling_grids[:, :, :, level].permute(
            0, 2, 1, 3, 4).flatten(0, 1)
        # N_*M_, D_, Lq_, P_
        sampling_value_l_ = F.grid_sample(
            value_l_,
            sampling_grid_l_,
            mode='bilinear',
            padding_mode='zeros',
            align_corners=False)
        sampling_value_list.append(sampling_value_l_)
    # (N_, Lq_, M_, L_, P_) -> (N_, M_, Lq_, L_, P_) -> (N_*M_, 1, Lq_, L_*P_)
    attention_weights = attention_weights.permute(0, 2, 1, 3, 4).reshape(
        bs * n_head, 1, Len_q, n_levels * n_points)
    output = (torch.stack(
        sampling_value_list, dim=-2).flatten(-2) *
              attention_weights).sum(-1).reshape(bs, n_head * c, Len_q)

    return output.permute(0, 2, 1)


class MSDeformableAttention(nn.Module):
    def __init__(self, embed_dim=256, num_heads=8, num_levels=4, num_points=4,):
        """
        Multi-Scale Deformable Attention Module
        """
        super(MSDeformableAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_levels = num_levels
        self.num_points = num_points
        self.total_points = num_heads * num_levels * num_points

        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        self.sampling_offsets = nn.Linear(embed_dim, self.total_points * 2,)
        self.attention_weights = nn.Linear(embed_dim, self.total_points)
        self.value_proj = nn.Linear(embed_dim, embed_dim)
        self.output_proj = nn.Linear(embed_dim, embed_dim)

        self.ms_deformable_attn_core = deformable_attention_core_func

        self._reset_parameters()


    def _reset_parameters(self):
        # sampling_offsets
        init.constant_(self.sampling_offsets.weight, 0)
        thetas = torch.arange(self.num_heads, dtype=torch.float32) * (2.0 * math.pi / self.num_heads)
        grid_init = torch.stack([thetas.cos(), thetas.sin()], -1)
        grid_init = grid_init / grid_init.abs().max(-1, keepdim=True).values
        grid_init = grid_init.reshape(self.num_heads, 1, 1, 2).tile([1, self.num_levels, self.num_points, 1])
        scaling = torch.arange(1, self.num_points + 1, dtype=torch.float32).reshape(1, 1, -1, 1)
        grid_init *= scaling
        self.sampling_offsets.bias.data[...] = grid_init.flatten()

        # attention_weights
        init.constant_(self.attention_weights.weight, 0)
        init.constant_(self.attention_weights.bias, 0)

        # proj
        init.xavier_uniform_(self.value_proj.weight)
        init.constant_(self.value_proj.bias, 0)
        init.xavier_uniform_(self.output_proj.weight)
        init.constant_(self.output_proj.bias, 0)


    def forward(self,
                query,
                reference_points,
                value,
                value_spatial_shapes,
                value_mask=None):
        """
        Args:
            query (Tensor): [bs, query_length, C]
            reference_points (Tensor): [bs, query_length, n_levels, 2], range in [0, 1], top-left (0,0),
                bottom-right (1, 1), including padding area
            value (Tensor): [bs, value_length, C]
            value_spatial_shapes (List): [n_levels, 2], [(H_0, W_0), (H_1, W_1), ..., (H_{L-1}, W_{L-1})]
            value_level_start_index (List): [n_levels], [0, H_0*W_0, H_0*W_0+H_1*W_1, ...]
            value_mask (Tensor): [bs, value_length], True for non-padding elements, False for padding elements

        Returns:
            output (Tensor): [bs, Length_{query}, C]
        """
        bs, Len_q = query.shape[:2]
        Len_v = value.shape[1]

        value = self.value_proj(value)
        if value_mask is not None:
            value_mask = value_mask.astype(value.dtype).unsqueeze(-1)
            value *= value_mask
        value = value.reshape(bs, Len_v, self.num_heads, self.head_dim)

        sampling_offsets = self.sampling_offsets(query).reshape(
            bs, Len_q, self.num_heads, self.num_levels, self.num_points, 2)
        attention_weights = self.attention_weights(query).reshape(
            bs, Len_q, self.num_heads, self.num_levels * self.num_points)
        attention_weights = F.softmax(attention_weights, dim=-1).reshape(
            bs, Len_q, self.num_heads, self.num_levels, self.num_points)

        if reference_points.shape[-1] == 2:
            offset_normalizer = torch.tensor(value_spatial_shapes)
            offset_normalizer = offset_normalizer.flip([1]).reshape(
                1, 1, 1, self.num_levels, 1, 2)
            sampling_locations = reference_points.reshape(
                bs, Len_q, 1, self.num_levels, 1, 2
            ) + sampling_offsets / offset_normalizer
        elif reference_points.shape[-1] == 4:
            sampling_locations = (
                reference_points[:, :, None, :, None, :2] + sampling_offsets /
                self.num_points * reference_points[:, :, None, :, None, 2:] * 0.5)
        else:
            raise ValueError(
                "Last dim of reference_points must be 2 or 4, but get {} instead.".
                format(reference_points.shape[-1]))

        output = self.ms_deformable_attn_core(value, value_spatial_shapes, sampling_locations, attention_weights)

        output = self.output_proj(output)

        return output

"""deformable_attention+ add & norm"""

# @title
class deformable_attention(nn.Module):

    def __init__(self, C_dim, n_head, n_levels, n_points=4, dropout=0.0):

        super().__init__()

        self.deform_attn = MSDeformableAttention(embed_dim=C_dim,
                                                  num_heads=n_head,
                                                  num_levels=n_levels,
                                                  num_points=n_points
                                                  )
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(C_dim)

    def add_pos_embedding(self, x: Tensor, pos: Tensor) -> Tensor:
        return x if pos is None else x + pos

    def forward(
        self,
        x_last,                 # (B, Len_q, C)
        reference_points,       # (B, Len_q, n_levels, 2 or 4)
        img_feat,               # (B, Len_v, C)
        spatial_shapes,         # (n_levels, 2)
        mask=None,
        query_pos=None,
    ):
        query = self.add_pos_embedding(x_last, query_pos)
        deformable_attened = self.deform_attn(query=query,reference_points=reference_points,value=img_feat,value_spatial_shapes=spatial_shapes,value_mask=mask, )
        x_next = x_last + self.dropout(deformable_attened)
        x_next = self.norm(x_next)

        return x_next

"""FFN+ add & norm"""

# @title
class FFN(nn.Module):
    #C_dim: feature dimension
    #dim_feedforward: FFN 내부의 hidden layer 차원
    #논문에서 dropout이 0일 때가 퍼포먼스가 가장 좋았다고 해서 default를 0으로 설정
    def __init__(self, C_dim, dim_feedforward=2048, dropout=0.0):
        super(FFN, self).__init__()

        self.first_linear = nn.Linear(C_dim, dim_feedforward)
        self.second_linear = nn.Linear(dim_feedforward, C_dim)

        self.dropout = nn.Dropout(dropout)

        self.activation = F.relu

        self.norm = nn.LayerNorm(C_dim)
        #parameter 초기화, xavier_uniform_ 사용
        self.reset_parameters()

    def reset_parameters(self):
        for par in self.parameters():
            if par.dim() > 1:
                nn.init.xavier_uniform_(par)

    def forward(self, x_last):
        # FFN
        # FFN = relu(w1x + b1)W2 + b2
        ffn = self.second_linear(self.dropout(self.activation(self.first_linear(x_last))))

        # add & norm
        # 논문에 나오는 add&norm 부분을 FFN 모듈 안에 구현
        # add: x + f(x) / norm : 정규화(LayerNorm)
        x_next = x_last + self.dropout(ffn)
        x_next = self.norm(x_next)

        return x_next

"""EFDDecoder"""

# @title
class EFDDecoder(nn.Module):

    def __init__(
        self,
        C_dim=256,
        n_head=8,
        dim_feedforward=1024,
        n_levels=3,
        n_points=4,
        dropout=0.0,
        num_classes=2,
        num_harmonics=1,
    ):
        super().__init__()

        self.self_attn = self_attention(
            C_dim=C_dim,
            n_head=n_head,
            dropout=dropout,
        )

        self.deform_attn = deformable_attention(
            C_dim=C_dim,
            n_head=n_head,
            n_levels=n_levels,
            n_points=n_points,
            dropout=dropout,
        )

        self.ffn = FFN(
            C_dim=C_dim,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )

        self.class_head = nn.Linear(C_dim, 1)  # binary (building / not)
        self.bbox_head = MLP(C_dim, C_dim, 4, num_layers=3)
        self.efd_head  = MLP(C_dim, C_dim, 4 * num_harmonics + 1, num_layers=2)

    def forward(
        self,
        tgt,                 # (B, Q_total, C)
        ref_points_unact,    # (B, Q_total, 4)
        memory,              # (B, S, C)
        spatial_shapes,      # (L, 2)
        query_pos_head,      # MLP(4→C)
        attn_mask=None,      # (Q_total, Q_total) or None
    ):
        """
        Q_total = DN 쿼리 + 메인 쿼리 개수
        """

        # 1) reference points → pos embedding
        ref_points = ref_points_unact.sigmoid()            # (B,Q,4)
        query_pos = query_pos_head(ref_points)            # (B,Q,C)

        # 2) self-attention (DN mask 반영 가능)
        x = self.self_attn(
            tgt,
            mask=attn_mask,        # ★ 여기서 attn_mask 사용 가능
            query_pos=query_pos,
        )

        # 3) deformable attention
        if torch.is_tensor(spatial_shapes):
            num_levels = spatial_shapes.shape[0]
        else:
            num_levels = len(spatial_shapes)

        ref_input = ref_points.unsqueeze(2).expand(-1, -1, num_levels, -1)

        x = self.deform_attn(
            x,               # x_last
            ref_input,       # reference_points
            memory,          # img_feat
            spatial_shapes,  # spatial_shapes
            mask=None,
            query_pos=query_pos,
        )

        # 4) FFN
        x = self.ffn(x)

        # 5) heads
        class_logits = self.class_head(x)         # (B,Q,1)

        delta_bbox = self.bbox_head(x)           # (B,Q,4)
        new_ref_unact = delta_bbox + ref_points_unact
        new_ref = new_ref_unact.sigmoid()        # (B,Q,4) in [0,1]

        efd_params = self.efd_head(x)            # (B,Q,4n+1)

        return x, new_ref_unact, new_ref, class_logits, efd_params

"""EFDinversetransform"""

# @title
class EFDinversetransform(nn.Module):
    """
    EFD 기반 곡선 복원 모듈.

    입력:
        efd_params: (B, Q, 4n+1)
            - 첫 번째 채널: scale (스칼라)
            - 나머지: [a1..an, b1..bn, c1..cn, d1..dn] 순서라고 가정
        boxes: (B, Q, 4) = (cx, cy, w, h)  [선택]
        local:
            True  → EFD 로컬 좌표 ([-1, 1] 근처)로 리턴
        map_to_box:
            True  → boxes 정보로 (cx,cy,w,h)에 맞게 매핑한 폴리곤 리턴

    출력:
        polys: (B, Q, T, 2)  where T = num_points
    """

    def __init__(self, num_harmonics: int = None, num_points: int = 128):
        super().__init__()
        self.num_harmonics = num_harmonics  # None이면 forward에서 자동 추론
        self.num_points = num_points

    def forward(
        self,
        efd_params: torch.Tensor,   # (B, Q, 4n+1)
        boxes: torch.Tensor = None, # (B, Q, 4)
        local: bool = False,
        map_to_box: bool = True,
    ) -> torch.Tensor:
        B, Q, D = efd_params.shape

        # ===== 0) harmonic 수 자동 추론 =====
        if self.num_harmonics is None:
            assert (D - 1) % 4 == 0, f"EFD params 마지막 차원 D={D} 는 4n+1 형태여야 합니다."
            n = (D - 1) // 4
        else:
            n = self.num_harmonics
            expected_D = 4 * n + 1
            assert D == expected_D, f"efd_params 마지막 차원 D={D} 는 4n+1 (={expected_D}) 이어야 합니다."

        device = efd_params.device
        dtype = efd_params.dtype

        # ===== 1) scale + 계수 분리 =====
        scale = efd_params[..., 0:1]        # (B, Q, 1)
        coeffs = efd_params[..., 1:]        # (B, Q, 4n)
        coeffs = coeffs.view(B, Q, n, 4)    # (B, Q, n, 4)

        # a_n, b_n, c_n, d_n  (각각 (B, Q, n))
        a, b, c, d = coeffs.unbind(dim=-1)

        # ===== 2) t 샘플링 (0 ~ 2π) =====
        #   t: (1, 1, 1, T)
        T = self.num_points
        t = torch.linspace(0, 2 * math.pi, T, device=device, dtype=dtype)
        t = t.view(1, 1, 1, T)

        #   n_idx: (1, 1, n, 1)  → n * t 브로드캐스트
        n_idx = torch.arange(1, n + 1, device=device, dtype=dtype).view(1, 1, n, 1)

        cos_nt = torch.cos(n_idx * t)   # (B=1, Q=1, n, T)
        sin_nt = torch.sin(n_idx * t)

        # (B, Q, n, 1) 로 확장
        a = a.unsqueeze(-1)
        b = b.unsqueeze(-1)
        c = c.unsqueeze(-1)
        d = d.unsqueeze(-1)

        # ===== 3) Fourier 역변환 (로컬 좌표계 곡선) =====
        # x(t) = Σ [ a_n cos(n t) + b_n sin(n t) ]
        # y(t) = Σ [ c_n cos(n t) + d_n sin(n t) ]
        x = (a * cos_nt + b * sin_nt).sum(dim=2)  # (B, Q, T)
        y = (c * cos_nt + d * sin_nt).sum(dim=2)  # (B, Q, T)

        # ===== 4) scale 적용 (global shape 조절) =====
        #   scale: (B, Q, 1) → 브로드캐스트
        x = x * scale
        y = y * scale

        # 현재 polys_local은 EFD 로컬 좌표 (centered, scale 적용됨)
        polys_local = torch.stack([x, y], dim=-1)  # (B, Q, T, 2)

        # ===== 5) 로컬 좌표로만 뽑고 싶을 때 =====
        if local and not map_to_box:
            # [-1, 1] 근처에 맞게 정규화 (gradient 유지)
            #   (x, y) 전체에서 최대 절댓값 기준으로 나눔
            max_val = polys_local.abs().amax(dim=2, keepdim=True).clamp(min=1e-4)
            polys_norm = polys_local / max_val
            return polys_norm  # (B, Q, T, 2)

        # ===== 6) boxes 기반으로 (cx,cy,w,h) 공간에 매핑 =====
        if map_to_box:
            assert boxes is not None, "map_to_box=True 인 경우 boxes (B,Q,4)가 필요합니다."
            assert boxes.shape[:2] == (B, Q), f"boxes shape={boxes.shape}, efd_params shape={efd_params.shape}"

            cx, cy, w, h = boxes.unbind(dim=-1)  # 각 (B, Q)

            # (B, Q, 1) 로 reshape 후 브로드캐스트
            cx = cx.unsqueeze(-1)
            cy = cy.unsqueeze(-1)
            w = w.unsqueeze(-1)
            h = h.unsqueeze(-1)

            # polys_local: (B, Q, T, 2)
            x_local = polys_local[..., 0]   # (B, Q, T)
            y_local = polys_local[..., 1]

            # [-1,1] 근처라고 보고 w/2, h/2 스케일로 박스에 매핑
            px = cx + x_local * (w / 2.0)
            py = cy + y_local * (h / 2.0)

            polys_box = torch.stack([px, py], dim=-1)  # (B, Q, T, 2)
            return polys_box

        # 기본값: scale까지 적용된 로컬 좌표 (박스 매핑 X)
        return polys_local

"""PolygonGroupSampler"""

# @title

class PolygonGroupSampler(nn.Module):
    """
    논문 EFDTR 기반 Polygon Decoder 전처리 모듈.
    EFD inverse transform으로 만든 128개 버텍스를
    group_size(기본 4)로 묶어 32개 그룹을 만들고
    각 그룹의 대표 xy(center)을 계산한다.

    입력:
        polys_box: (B, Q, T, 2)  — 128 vertices
    출력:
        polys_grouped: (B, Q, num_groups, group_size, 2)
        group_centers: (B, Q, num_groups, 2)
    """

    def __init__(self, group_size=4):
        super().__init__()
        self.group_size = group_size

    def forward(self, polys_box):
        """
        polys_box: (B, Q, T, 2)
        """
        B, Q, T, _ = polys_box.shape
        g = self.group_size

        assert T % g == 0, f"Vertex count T={T} must be divisible by group_size={g}"
        num_groups = T // g  # 128/4 = 32

        # (B, Q, 128, 2) → (B, Q, 32, 4, 2)
        polys_grouped = polys_box.view(B, Q, num_groups, g, 2)

        # group centroid: (B, Q, 32, 2)
        group_centers = polys_grouped.mean(dim=3)

        return polys_grouped, group_centers

"""MultiScaleFeatureSampler"""

# @title
class MultiScaleFeatureSampler(nn.Module):
    """
    EFDTR 스타일 멀티스케일 피처 샘플러.
    - 32개 그룹 중심점(group_centers)을 이용해 P2~P5에서 grid_sample
    - EFD 계수 + class logits로 4-scale fusion weight 예측
    - weighted sum으로 단일 스케일 쿼리 피처 생성

    Args:
        in_channels:   각 피처맵 채널 수 (모든 scale에서 동일하다고 가정, C)
        num_levels:    멀티 스케일 피처 개수 (기본 4: P2~P5)
        efd_dim:       EFD 계수 차원 (4n+1)
        cls_dim:       class 로짓 차원 (num_classes 또는 1)
        hidden_dim:    weight 예측용 MLP hidden dim
        out_dim:       최종 출력 채널 수 (None이면 in_channels 유지)
    """

    def __init__(
        self,
        in_channels: int,
        num_levels: int = 4,
        efd_dim: int = 5,
        cls_dim: int = 2,
        hidden_dim: int = 128,
        out_dim: int = None,
    ):
        super().__init__()
        self.in_channels = in_channels
        self.num_levels = num_levels
        self.efd_dim = efd_dim
        self.cls_dim = cls_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim or in_channels

        cond_dim = efd_dim + cls_dim  # [EFD, class] concat

        # EFD + class → 4-scale weight 예측 MLP
        self.scale_mlp = nn.Sequential(
            nn.Linear(cond_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, num_levels),
        )

        # 채널 축소/변환 (optional)
        self.output_proj = nn.Linear(in_channels, self.out_dim)

    def forward(
        self,
        multi_scale_feats,
        group_centers: torch.Tensor,  # (B, Q, G, 2) in [0,1]
        efd_params: torch.Tensor,     # (B, Q, D_efd)
        class_logits: torch.Tensor,   # (B, Q, D_cls)
    ):
        """
        Args:
            multi_scale_feats: list of length L=num_levels
                each tensor: (B, C, H_l, W_l)
            group_centers: (B, Q, G, 2) - normalized [0,1] coords
            efd_params:    (B, Q, D_efd)
            class_logits:  (B, Q, D_cls)

        Returns:
            fused_feats: (B, Q, G, C_out)
        """
        assert len(multi_scale_feats) == self.num_levels, \
            f"Expected {self.num_levels} feature levels, got {len(multi_scale_feats)}"

        B, Q, G, _ = group_centers.shape
        C = self.in_channels
        device = group_centers.device
        dtype = group_centers.dtype

        # ----- 1) group_centers [0,1] → grid_sample 좌표 [-1,1] -----
        # grid_sample expects (N, H_out, W_out, 2) with coords in [-1, 1]
        # 여기서 H_out = G, W_out = 1 로 사용
        x = group_centers[..., 0]  # (B, Q, G)
        y = group_centers[..., 1]  # (B, Q, G)

        grid_x = x * 2.0 - 1.0
        grid_y = y * 2.0 - 1.0
        grid = torch.stack([grid_x, grid_y], dim=-1)  # (B, Q, G, 2)

        # (B, Q, G, 2) → (B*Q, G, 1, 2)
        grid_flat = grid.view(B * Q, G, 1, 2)

        # ----- 2) 각 scale에서 group_centers 기준으로 grid_sample -----
        level_feats = []  # list of (B, Q, G, C)

        for lvl, feat in enumerate(multi_scale_feats):
            # feat: (B, C, H_l, W_l) → (B*Q, C, H_l, W_l)
            assert feat.shape[0] == B, "Batch size mismatch between feats and group_centers"
            assert feat.shape[1] == C, "in_channels mismatch with feat channels"

            Bf, Cf, Hf, Wf = feat.shape

            # Q 차원 확장: (B, 1, C, H, W) → (B, Q, C, H, W) → (B*Q, C, H, W)
            feat_expanded = feat.unsqueeze(1).expand(Bf, Q, Cf, Hf, Wf)
            feat_flat = feat_expanded.reshape(Bf * Q, Cf, Hf, Wf)

            # grid_sample: (B*Q, C, G, 1)
            sampled = F.grid_sample(
                feat_flat,      # (B*Q, C, H, W)
                grid_flat,      # (B*Q, G, 1, 2)
                mode="bilinear",
                padding_mode="zeros",
                align_corners=False,
            )

            # (B*Q, C, G, 1) → (B, Q, C, G) → (B, Q, G, C)
            sampled = sampled.squeeze(-1)               # (B*Q, C, G)
            sampled = sampled.view(B, Q, Cf, G)         # (B, Q, C, G)
            sampled = sampled.permute(0, 1, 3, 2)       # (B, Q, G, C)

            level_feats.append(sampled)

        # ----- 3) EFD + class로 4-scale fusion weight 예측 -----
        # cond: (B, Q, cond_dim) → broadcast to (B, Q, G, cond_dim)
        cond = torch.cat([efd_params, class_logits], dim=-1)  # (B, Q, D_efd + D_cls)
        cond = cond.unsqueeze(2).expand(B, Q, G, cond.shape[-1])  # (B, Q, G, cond_dim)

        # (B, Q, G, L)
        scale_logits = self.scale_mlp(cond)
        scale_weights = F.softmax(scale_logits, dim=-1)  # normalize over levels

        # ----- 4) level_feats stack & weighted sum -----
        # level_feats: list of L tensors each (B, Q, G, C)
        # → stacked: (B, Q, G, L, C)
        stacked = torch.stack(level_feats, dim=3)

        # scale_weights: (B, Q, G, L) → (B, Q, G, L, 1)
        w = scale_weights.unsqueeze(-1)

        # weighted sum over levels L
        fused = (stacked * w).sum(dim=3)  # (B, Q, G, C)

        # ----- 5) optional channel projection -----
        fused = self.output_proj(fused)   # (B, Q, G, C_out)

        return fused  # Polygon Decoder 쿼리 피처

"""PolygonDecoder"""

# @title
class PolygonDecoder(nn.Module):

    def __init__(
        self,
        C_dim=256,
        n_head=8,
        dim_feedforward=1024,
        n_levels=4,
        n_points=4,
        dropout=0.0,
    ):
        super().__init__()

        self.self_attn = self_attention(
            C_dim=C_dim,
            n_head=n_head,
            dropout=dropout,
        )

        self.deform_attn = deformable_attention(
            C_dim=C_dim,
            n_head=n_head,
            n_levels=n_levels,
            n_points=n_points,
            dropout=dropout,
        )

        self.ffn = FFN(
            C_dim=C_dim,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )

        self.vertex_head = nn.Linear(C_dim, n_points * 2)

    def forward(
        self,
        tgt,               # (B, Q, G, C)
        reference_points,  # (B, Q, G, 2)  group center (normalized)
        polys_grouped,     # (B, Q, G, n_points, 2)
        memory,            # (B, S, C)
        spatial_shapes,    # (L,2)
        query_pos=None,    # unused or optional
        attn_mask=None,    # (Q_total, Q_total) or None
    ):
        B, Q, G, C = tgt.shape

        # 1) flatten group dimension
        tgt_flat = tgt.view(B, Q * G, C)
        ref = reference_points.view(B, Q * G, 1, 2)

        if query_pos is not None:
            query_pos_flat = query_pos.view(B, Q * G, C)
        else:
            query_pos_flat = None

        if torch.is_tensor(spatial_shapes):
            num_levels = spatial_shapes.shape[0]
        else:
            num_levels = len(spatial_shapes)

        # 2) self-attn
        x = self.self_attn(
            tgt_flat,
            mask=None,
            query_pos=query_pos_flat,
        )

        # 3) deformable attn
        x = self.deform_attn(
            x,
            ref.expand(-1, -1, num_levels, -1),
            memory,
            spatial_shapes,
            mask=None,
            query_pos=query_pos_flat,
        )

        x = x.view(B, Q, G, C)

        # 4) FFN
        x = self.ffn(x)

        # 5) vertex offsets
        v = self.vertex_head(x)          # (B,Q,G,2*n_points)
        v = v.view(B, Q, G, -1, 2)       # (B,Q,G,n_points,2)

        refined_vertices = polys_grouped + v

        return x, refined_vertices

"""EFDTRTransformerDecoder"""

# @title
class EFDTRTransformerDecoder(nn.Module):
    """
    전체 EFDTR-style Transformer Decoder (DN 완전 적용 버전)

    구성:
      1) RT-DETR-style top-k query selection (_get_decoder_input)
      2) EFDDecoder stage  (bbox + EFD + class 예측)
      3) EFD inverse transform → 128 vertices
      4) PolygonGroupSampler → 32 groups + group centers
      5) MultiScaleFeatureSampler → 멀티스케일 피처 + 4-scale weight fusion
      6) PolygonDecoder (multi-layer) → 128 vertices iterative refine
      7) Denoising (Contrastive Denoising, CDN) 완전 통합
    """

    def __init__(
        self,
        num_queries=300,
        num_classes=2,
        C_dim=256,
        n_head=8,
        dim_feedforward=1024,
        num_efd_layers=1,
        num_poly_layers=3,
        num_levels_enc=3,
        num_levels_ms=4,
        num_harmonics=1,
        num_points_poly=128,
        group_size=4,
        dropout=0.0,
        # DN 관련
        num_denoising=100,
        label_noise_ratio=0.5,
        box_noise_scale=1.0,
    ):
        super().__init__()

        self.num_queries = num_queries
        self.num_classes = num_classes
        self.C_dim = C_dim
        self.num_harmonics = num_harmonics
        self.num_points_poly = num_points_poly
        self.group_size = group_size

        self.num_denoising = num_denoising
        self.label_noise_ratio = label_noise_ratio
        self.box_noise_scale = box_noise_scale

        # 0) Denoising 쿼리 임베딩 (RT-DETR와 동일 방식)
        if self.num_denoising > 0:
            # num_classes+1: 마지막 인덱스를 padding용으로 사용
            self.denoising_class_embed = nn.Embedding(
                num_classes + 1, C_dim, padding_idx=num_classes
            )
        else:
            self.denoising_class_embed = None

        # 1) RT-DETR top-k decoder input (anchors, encoder feat → query)
        self.dec_input = _get_decoder_input(
            hidden_dim=C_dim,
            num_queries=num_queries,
            num_classes=num_classes,
            learnt_init_query=False,
        )

        # 2) EFD Decoder layers
        self.efd_layers = nn.ModuleList([
            EFDDecoder(
                C_dim=C_dim,
                n_head=n_head,
                dim_feedforward=dim_feedforward,
                n_levels=num_levels_enc,
                n_points=4,
                dropout=dropout,
                num_classes=num_classes,
                num_harmonics=num_harmonics,
            )
            for _ in range(num_efd_layers)
        ])

        # bbox → query_pos 변환용 MLP
        self.query_pos_head = MLP(
            input_dim=4,
            hidden_dim=C_dim * 2,
            output_dim=C_dim,
            num_layers=2,
        )

        # 3) EFD inverse transform
        self.efd_inverse = EFDinversetransform(
            num_harmonics=num_harmonics,
            num_points=num_points_poly,
        )

        # 4) group sampler (128 → 32×4)
        self.group_sampler = PolygonGroupSampler(group_size=group_size)

        # 5) multi-scale sampler (P0~P3)
        efd_dim = 4 * num_harmonics + 1
        cls_dim = 1  # EFDDecoder class_head output dim

        self.ms_sampler = MultiScaleFeatureSampler(
            in_channels=C_dim,
            num_levels=num_levels_ms,
            efd_dim=efd_dim,
            cls_dim=cls_dim,
            hidden_dim=128,
            out_dim=C_dim,
        )

        # 6) Polygon Decoder layers (iterative refine)
        self.polygon_layers = nn.ModuleList([
            PolygonDecoder(
                C_dim=C_dim,
                n_head=n_head,
                dim_feedforward=dim_feedforward,
                n_levels=num_levels_enc,
                n_points=group_size,   # group_size=4 → vertex_head out dim=8
                dropout=dropout,
            )
            for _ in range(num_poly_layers)
        ])

    def _prepare_denoising(self, targets):
        """
        RT-DETR의 get_contrastive_denoising_training_group를 호출.
        """
        if (not self.training) or (self.num_denoising <= 0) or (targets is None):
            return None, None, None, None

        dn_class, dn_bbox_unact, attn_mask, dn_meta = \
            get_contrastive_denoising_training_group(
                targets=targets,
                num_classes=self.num_classes,
                num_queries=self.num_queries,
                class_embed=self.denoising_class_embed,
                num_denoising=self.num_denoising,
                label_noise_ratio=self.label_noise_ratio,
                box_noise_scale=self.box_noise_scale,
            )
        return dn_class, dn_bbox_unact, attn_mask, dn_meta

    def forward(
        self,
        memory,            # (B, S, C)  from _get_encoder_input(pixel_outs_3)
        spatial_shapes,    # python list or tensor [(H1,W1), (H2,W2), ...]
        anchors,           # (B, S, 4)
        valid_mask,        # (B, S, 1)
        multi_scale_feats, # list of feature maps for ms sampling (e.g., pixel_outs)
        targets=None,      # 학습 시 GT (DN 용)
    ):
        """
        Returns (train mode):
            out_main:
              {
                'class_logits': (B, Q, 1),
                'boxes':        (B, Q, 4),
                'efd_params':   (B, Q, 4n+1),
                'polygons':     (B, Q, 128, 2),
              }
            out_dn:
              {
                'class_logits': (B, K, 1),
                'boxes':        (B, K, 4),
                'efd_params':   (B, K, 4n+1),
                'polygons':     (B, K, 128, 2),
              }
            dn_meta: DN 관련 메타 (dn_num_split 등)

        Returns (eval mode):
            out_main만 반환
        """
        device = memory.device
        if not torch.is_tensor(spatial_shapes):
            spatial_shapes_tensor = torch.as_tensor(
                spatial_shapes, device=device, dtype=torch.long
            )
        else:
            spatial_shapes_tensor = spatial_shapes.to(device)

        # --------------------------------------------------
        # 0) DN 쿼리 생성 (training+num_denoising>0+targets 있을 때만)
        # --------------------------------------------------
        dn_class, dn_bbox_unact, attn_mask, dn_meta = \
            self._prepare_denoising(targets)

        # --------------------------------------------------
        # 1) RT-DETR top-k query selection (+ DN 붙이기)
        # --------------------------------------------------
        target, ref_points_unact, topk_bboxes, topk_logits = self.dec_input(
            memory=memory,
            anchors=anchors,
            valid_mask=valid_mask,
            denoising_class=dn_class,
            denoising_bbox_unact=dn_bbox_unact,
        )
        # target:         (B, K+Q, C)
        # ref_points_unact: (B, K+Q, 4)

        # --------------------------------------------------
        # 2) EFD Decoder stage (iterative)
        # --------------------------------------------------
        x = target
        ref_unact = ref_points_unact
        class_logits = None
        efd_params = None
        ref_sigmoid = None

        for layer in self.efd_layers:
            x, ref_unact, ref_sigmoid, class_logits, efd_params = layer(
                tgt=x,
                ref_points_unact=ref_unact,
                memory=memory,
                spatial_shapes=spatial_shapes_tensor,
                query_pos_head=self.query_pos_head,
                attn_mask=attn_mask,             # ★ CDN 마스크 전달 가능
            )
            ref_unact = ref_unact

        # --------------------------------------------------
        # 3) EFD inverse → 128 vertices
        # --------------------------------------------------
        polys_box = self.efd_inverse(
            efd_params,       # (B, K+Q, 4n+1)
            boxes=ref_sigmoid,
            local=False,
            map_to_box=True,
        )  # (B, K+Q, 128, 2)

        # --------------------------------------------------
        # 4) Group sampling
        # --------------------------------------------------
        polys_grouped, group_centers = self.group_sampler(polys_box)
        # polys_grouped: (B, K+Q, G, 4, 2)
        # group_centers: (B, K+Q, G, 2)

        # --------------------------------------------------
        # 5) Multi-scale feature sampling + fusion
        # --------------------------------------------------
        fused_queries = self.ms_sampler(
            multi_scale_feats=multi_scale_feats,
            group_centers=group_centers,
            efd_params=efd_params,
            class_logits=class_logits,
        )  # (B, K+Q, G, C)

        # --------------------------------------------------
        # 6) PolygonDecoder (multi-layer iterative refine)
        # --------------------------------------------------
        tgt_poly = fused_queries          # (B,K+Q,G,C)
        ref_pts = group_centers           # (B,K+Q,G,2)
        polys_grp = polys_grouped         # (B,K+Q,G,4,2)

        for layer in self.polygon_layers:
            tgt_poly, polys_grp = layer(
                tgt=tgt_poly,
                reference_points=ref_pts,
                polys_grouped=polys_grp,
                memory=memory,
                spatial_shapes=spatial_shapes_tensor,
                query_pos=None,
                attn_mask=None,     # ★ 여기도 None
            )
            ref_pts = polys_grp.mean(dim=3)  # (B,K+Q,G,2)

        final_polys = polys_grp.reshape(
            polys_grp.shape[0], polys_grp.shape[1], -1, 2
        )  # (B, K+Q, 128, 2)

        # --------------------------------------------------
        # 7) DN / Main Query split
        # --------------------------------------------------
        if (self.training) and (dn_meta is not None):
            dn_num, main_num = dn_meta["dn_num_split"]  # [K, Q]
            # K+Q 확인용 assert
            assert class_logits.shape[1] == dn_num + main_num

            # 앞 K = DN, 뒤 Q = main
            cls_dn   = class_logits[:, :dn_num]
            box_dn   = ref_sigmoid[:, :dn_num]
            efd_dn   = efd_params[:, :dn_num]
            poly_dn  = final_polys[:, :dn_num]

            cls_main  = class_logits[:, dn_num:]
            box_main  = ref_sigmoid[:, dn_num:]
            efd_main  = efd_params[:, dn_num:]
            poly_main = final_polys[:, dn_num:]

            out_main = {
                "class_logits": cls_main,
                "boxes":        box_main,
                "efd_params":   efd_main,
                "polygons":     poly_main,
            }
            out_dn = {
                "class_logits": cls_dn,
                "boxes":        box_dn,
                "efd_params":   efd_dn,
                "polygons":     poly_dn,
            }
            return out_main, out_dn, dn_meta

        else:
            # eval 모드 or DN 사용 안할 때는 main만 (K+Q 전체를 main 취급)
            out = {
                "class_logits": class_logits,
                "boxes":        ref_sigmoid,
                "efd_params":   efd_params,
                "polygons":     final_polys,
            }
            return out
# EFDTRModel

class EFDTRModel(nn.Module):
    """
    D2Swin 백본 + MSDeform Pixel Decoder + EFDTR Transformer Decoder
    전부 한 번에 묶은 최종 모델 래퍼.

    forward(images, targets=None) 으로 바로 호출 가능.

    Args:
        backbone_name: timm Swin 모델 이름
        num_classes:   클래스 수 (건물=1 + 배경)
        num_queries:   detection query 개수 (RT-DETR top-k)
        num_denoising: DN query 개수 (0이면 DN 비활성)
    """

    def __init__(
        self,
        backbone_name: str = "swin_base_patch4_window7_224",
        num_classes: int = 2,
        num_queries: int = 100,
        num_denoising: int = 20,
        C_dim: int = 256,
        num_efd_layers: int = 2,
        num_poly_layers: int = 2,
        num_levels_enc: int = 3,   # encoder에 쓰는 pixel_out level 수 (P0~P2)
        num_levels_ms: int = 4,    # multi-scale sampler에 쓰는 level 수 (P0~P3)
        num_harmonics: int = 1,
        num_points_poly: int = 128,
        group_size: int = 4,
    ):
        super().__init__()

        self.num_classes = num_classes
        self.num_queries = num_queries
        self.num_denoising = num_denoising

        # --------------------------------------------------
        # 1) Backbone
        # --------------------------------------------------
        self.backbone = D2SwinTransformer(model_name=backbone_name)

        # --------------------------------------------------
        # 2) Pixel Decoder (Mask2Former MSDeformAttention PixelDecoder)
        # --------------------------------------------------
        # feature_channels는 backbone 출력 채널과 맞춰야 함
        # 현재 D2SwinTransformer: res2=128, res3=256, res4=512, res5=1024
        self.pixel_decoder = MSDeform_attention_pixel_decoder(
            feature_channels=[128, 256, 512, 1024]
        )

        # --------------------------------------------------
        # 3) Encoder Input / Anchors 생성 모듈
        # --------------------------------------------------
        self.encoder_input = _get_encoder_input()
        self.anchor_gen = _generate_anchors(
            eps=1e-2,
            feat_strides=[4, 8, 16, 32],   # spatial_shapes를 넘겨줄 거라 큰 의미는 없음
            eval_spatial_size=[224, 224],  # spatial_shapes=None일 때만 사용
        )

        # --------------------------------------------------
        # 4) EFDTR Transformer Decoder (DN 내장)
        # --------------------------------------------------
        self.decoder = EFDTRTransformerDecoder(
            num_queries=num_queries,
            num_classes=num_classes,
            C_dim=C_dim,
            n_head=8,
            dim_feedforward=1024,
            num_efd_layers=num_efd_layers,
            num_poly_layers=num_poly_layers,
            num_levels_enc=num_levels_enc,
            num_levels_ms=num_levels_ms,
            num_harmonics=num_harmonics,
            num_points_poly=num_points_poly,
            group_size=group_size,
            dropout=0.0,
            num_denoising=num_denoising,
            label_noise_ratio=0.5,
            box_noise_scale=0.4,
        )

    def forward(self, images: torch.Tensor, targets=None):
        """
        Args:
            images:  (B,3,H,W) 입력 이미지
            targets: 학습 시 GT 리스트
                [
                  {
                    "boxes":  (Ni,4)  cx,cy,w,h in [0,1]
                    "labels": (Ni,)   class index
                    ...
                  },
                  ...
                ]

        Returns:
            train 모드 & DN ON:
                out_main, out_dn, dn_meta
            그 외:
                out  (decoder에서 나온 dict 그대로)
        """
        device = images.device

        # --------------------------------------------------
        # 1) Backbone
        # --------------------------------------------------
        feats = self.backbone(images)
        # feats: {"res2":(B,128,H/4,W/4), ..., "res5":(B,1024,H/32,W/32)}

        # --------------------------------------------------
        # 2) Pixel Decoder
        # --------------------------------------------------
        pixel_outs = self.pixel_decoder.forward_features(feats)
        # pixel_outs: [P0, P1, P2, P3]
        # P0: (B,256,H/32,W/32)
        # P1: (B,256,H/16,W/16)
        # P2: (B,256,H/8, W/8)
        # P3: (B,256,H/4, W/4)

        # --------------------------------------------------
        # 3) Encoder Input 만들기 (RT-DETR-style)
        #    여기서는 P0~P2, 즉 1/32, 1/16, 1/8만 사용
        # --------------------------------------------------
        memory, spatial_shapes, level_start_index = self.encoder_input(pixel_outs[:3])
        # memory: (B, S, C=256)
        # spatial_shapes: [[H0,W0],[H1,W1],[H2,W2]]
        # level_start_index: [0, H0*W0, H0*W0+H1*W1]

        # spatial_shapes를 tensor/리스트 둘 다 대응
        if torch.is_tensor(spatial_shapes):
            spatial_shapes_list = spatial_shapes.tolist()
            spatial_shapes_tensor = spatial_shapes.to(device=device, dtype=torch.long)
        else:
            spatial_shapes_list = spatial_shapes
            spatial_shapes_tensor = torch.as_tensor(
                spatial_shapes_list, dtype=torch.long, device=device
            )

        # --------------------------------------------------
        # 4) Anchors + valid_mask 생성
        # --------------------------------------------------
        anchors_single, valid_mask_single = self.anchor_gen(
            spatial_shapes=spatial_shapes_list,
            device=device,
        )
        # anchors_single: (1, S, 4)
        # valid_mask_single: (1, S, 1)

        B, S, C = memory.shape
        anchors = anchors_single.expand(B, -1, -1)      # (B,S,4)
        valid_mask = valid_mask_single.expand(B, -1, -1)  # (B,S,1)

        # --------------------------------------------------
        # 5) EFDTR Transformer Decoder (DN 포함)
        # --------------------------------------------------
        # decoder는 학습 모드 + targets 있을 때 DN을 켠다고 가정
        if self.training:
            outputs = self.decoder(
                memory=memory,
                spatial_shapes=spatial_shapes_tensor,
                anchors=anchors,
                valid_mask=valid_mask,
                multi_scale_feats=pixel_outs,
                targets=targets,     # DN용
            )
        else:
            # 평가/추론일 때는 DN 없이
            outputs = self.decoder(
                memory=memory,
                spatial_shapes=spatial_shapes_tensor,
                anchors=anchors,
                valid_mask=valid_mask,
                multi_scale_feats=pixel_outs,
                targets=None,
            )

        # decoder가 (out_main, out_dn, dn_meta) 튜플을 리턴하는 경우 / dict 하나만 리턴하는 경우 둘 다 지원
        return outputs


Matcher

In [ ]:
# @title
def box_iou(boxes1, boxes2):
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)
    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]
    wh = (rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]
    union = area1[:, None] + area2 - inter
    iou = inter / union
    return iou, union

def generalized_box_iou(boxes1, boxes2):
    assert (boxes1[:, 2:] >= boxes1[:, :2]).all()
    assert (boxes2[:, 2:] >= boxes2[:, :2]).all()
    iou, union = box_iou(boxes1, boxes2)
    lt = torch.min(boxes1[:, None, :2], boxes2[:, :2])
    rb = torch.max(boxes1[:, None, 2:], boxes2[:, 2:])
    wh = (rb - lt).clamp(min=0)  # [N,M,2]
    area = wh[:, :, 0] * wh[:, :, 1]
    return iou - (area - union) / area

class HungarianMatcher(nn.Module):
    """This class computes an assignment between the targets and the predictions of the network

    For efficiency reasons, the targets don't include the no_object. Because of this, in general,
    there are more predictions than targets. In this case, we do a 1-to-1 matching of the best predictions,
    while the others are un-matched (and thus treated as non-objects).
    """

    __share__ = ['use_focal_loss', ]

    def __init__(self, weight_dict, use_focal_loss=False, alpha=0.25, gamma=2.0):
        """Creates the matcher

        Params:
            cost_class: This is the relative weight of the classification error in the matching cost
            cost_bbox: This is the relative weight of the L1 error of the bounding box coordinates in the matching cost
            cost_giou: This is the relative weight of the giou loss of the bounding box in the matching cost
        """
        super().__init__()
        self.cost_class = weight_dict['cost_class']
        self.cost_bbox = weight_dict['cost_bbox']
        self.cost_giou = weight_dict['cost_giou']

        self.use_focal_loss = use_focal_loss
        self.alpha = alpha
        self.gamma = gamma

        assert self.cost_class != 0 or self.cost_bbox != 0 or self.cost_giou != 0, "all costs cant be 0"
    @torch.no_grad()
    def forward(self, outputs, targets):
        bs, num_queries = outputs["pred_logits"].shape[:2]

        # 1) 시그모이드 확률 (B*Q, 1)
        logits = outputs["pred_logits"].flatten(0, 1)      # [B*Q, 1]
        out_prob = logits.sigmoid().squeeze(-1)           # [B*Q]

        out_bbox = outputs["pred_boxes"].flatten(0, 1)    # [B*Q, 4]

        # 2) 타겟 모으기
        #   labels는 어차피 전부 1(건물)이라고 가정
        tgt_bbox = torch.cat([v["boxes"] for v in targets])
        sizes = [len(v["boxes"]) for v in targets]        # 각 batch당 GT 개수

        # 3) classification cost:
        #   모든 GT는 "건물"이라고 생각하고,
        #   예측 확률이 높을수록 cost를 낮게 (-p)로 설정
        #   (B*Q, 1) → (B*Q, num_total_gt)
        cost_class = -out_prob.unsqueeze(1).expand(-1, tgt_bbox.shape[0])

        # 4) bbox L1 cost
        cost_bbox = torch.cdist(out_bbox, tgt_bbox, p=1)

        # 5) GIoU cost
        cost_giou = -generalized_box_iou(
            box_cxcywh_to_xyxy(out_bbox),
            box_cxcywh_to_xyxy(tgt_bbox),
        )

        # 6) 최종 cost
        C = self.cost_bbox * cost_bbox + self.cost_class * cost_class + self.cost_giou * cost_giou
        C = C.view(bs, num_queries, -1).cpu()

        indices = [linear_sum_assignment(c[i]) for i, c in enumerate(C.split(sizes, -1))]
        return [
            (torch.as_tensor(i, dtype=torch.int64),
            torch.as_tensor(j, dtype=torch.int64))
            for i, j in indices
        ]

Criterion

In [ ]:
# @title
class SetCriterion(nn.Module):
    """
    Project-Final unified Criterion:
    - vfl: classification
    - boxes: L1 + GIoU (same as RT-DETR)
    - efd: L1 regression of Fourier coefficients
    - polygon: phase-aligned polygon regression (EFDTR 3.4)

    Only these 4 losses exist.
    """

    def __init__(self, matcher, weight_dict, losses,
                 alpha=0.2, gamma=2.0, eos_coef=1e-4, num_classes=80):
        super().__init__()
        self.num_classes = num_classes
        self.matcher = matcher
        self.weight_dict = weight_dict
        self.losses = losses
        self.alpha = alpha
        self.gamma = gamma

        empty_weight = torch.ones(num_classes + 1)
        empty_weight[-1] = eos_coef
        self.register_buffer("empty_weight", empty_weight)

    # ------------------------------------------------
    # Utility: src indices for matched queries
    # ------------------------------------------------
    def _get_src_permutation_idx(self, indices):
        batch_idx = torch.cat([
            torch.full_like(src, b, dtype=torch.int64)
            for b, (src, _) in enumerate(indices)
        ])
        src_idx = torch.cat([src for (src, _) in indices])
        return batch_idx, src_idx

    # ============================================
    # 1) VFL Classification loss
    # ============================================
    def loss_vfl(self, outputs, targets, indices, num_boxes, log=True):
        assert 'pred_logits' in outputs and 'pred_boxes' in outputs

        src_logits = outputs["pred_logits"]     # (B, Q, C)
        src_boxes  = outputs["pred_boxes"]      # (B, Q, 4)

        idx = self._get_src_permutation_idx(indices)
        matched_pred_boxes = src_boxes[idx]

        # GT boxes
        target_boxes = torch.cat(
            [t["boxes"][J] for t, (_, J) in zip(targets, indices)],
            dim=0
        )

        # IoU as supervision score
        from torchvision.ops import box_iou
        iou, _ = box_iou(box_cxcywh_to_xyxy(matched_pred_boxes),
                         box_cxcywh_to_xyxy(target_boxes))
        iou_diag = torch.diag(iou).detach()

        # GT label map
        target_classes_o = torch.cat([
            t["labels"][J] for t, (_, J) in zip(targets, indices)
        ])
        target_classes = torch.full(
            src_logits.shape[:2], self.num_classes,
            dtype=torch.int64, device=src_logits.device
        )
        target_classes[idx] = target_classes_o

        # One-hot & score
        target_onehot = F.one_hot(target_classes, self.num_classes + 1)[..., :-1]
        score_map = torch.zeros_like(target_classes, dtype=src_logits.dtype)
        score_map[idx] = iou_diag
        score_map = score_map.unsqueeze(-1) * target_onehot

        pred_sigmoid = torch.sigmoid(src_logits).detach()
        weight = (
            self.alpha * pred_sigmoid.pow(self.gamma) * (1 - target_onehot)
            + score_map
        )

        loss = F.binary_cross_entropy_with_logits(
            src_logits, score_map, weight=weight, reduction="none"
        )
        loss = loss.mean(1).sum() / max(num_boxes, 1)

        return {"loss_vfl": loss}

    # ============================================
    # 2) Box regression loss (L1 + GIoU)
    # ============================================
    def loss_boxes(self, outputs, targets, indices, num_boxes, log=True):
        assert 'pred_boxes' in outputs

        idx = self._get_src_permutation_idx(indices)
        src_boxes = outputs['pred_boxes'][idx]

        target_boxes = torch.cat(
            [t["boxes"][J] for t, (_, J) in zip(targets, indices)], dim=0
        )

        # L1
        loss_l1 = F.l1_loss(src_boxes, target_boxes, reduction='none').sum()

        # GIoU
        from torchvision.ops import generalized_box_iou
        giou = generalized_box_iou(box_cxcywh_to_xyxy(src_boxes),
                                   box_cxcywh_to_xyxy(target_boxes))
        loss_giou = (1 - torch.diag(giou)).sum()

        loss = loss_l1 / num_boxes + loss_giou / num_boxes

        return {
            "loss_bbox": loss_l1 / num_boxes,
            "loss_giou": loss_giou / num_boxes,
        }

    # ============================================
    # 🔥 EFDTR EXTENSION PART
    # 3) EFD L1 Loss
    # 4) Phase-aligned Polygon Loss
    # ============================================

    # ---------- arc length ----------
    @staticmethod
    def _compute_arc_length_t(poly):
        next_pts = torch.roll(poly, -1, dims=0)
        seg = (next_pts - poly).norm(dim=-1)
        return torch.cat([
            torch.zeros(1, device=poly.device, dtype=poly.dtype),
            seg.cumsum(0)
        ])  # (P+1)

    @staticmethod
    def _phases_from_t(t):
        return 2 * math.pi * t[:-1] / t[-1].clamp(min=1e-6)

    @staticmethod
    def _snap(pred_phi, gt_phi, n_pred):
        diff = (pred_phi[:, None] - gt_phi[None, :]).abs()
        diff = torch.minimum(diff, 2 * math.pi - diff)
        min_val, min_idx = diff.min(dim=1)
        threshold = math.pi / n_pred
        snapped = pred_phi.clone()
        mask = min_val < threshold
        snapped[mask] = gt_phi[min_idx[mask]]
        return snapped

    # ---------- interpolate polygon ----------
    @staticmethod
    def _interp(poly, t, phases):
        T = t[-1]
        t_phase = phases * T / (2 * math.pi)

        idx = torch.searchsorted(t, t_phase, right=True) - 1
        idx = idx.clamp(0, poly.size(0) - 1)

        t0 = t[idx]
        t1 = t[idx + 1]
        w = ((t_phase - t0) / (t1 - t0).clamp(min=1e-6)).unsqueeze(-1)

        p0 = poly[idx]
        p1 = poly[(idx + 1) % poly.size(0)]
        return (1 - w) * p0 + w * p1

    # ---------- full polygon loss ----------
    def loss_polygon(self, outputs, targets, indices, num_boxes, log=True):
        pred_polys = outputs['pred_polygons']    # (B, Q, P, 2)
        all_losses = []

        for b, (src_idx, tgt_idx) in enumerate(indices):
            if len(src_idx) == 0:
                continue

            pred_list = [pred_polys[b, s] for s in src_idx]
            gt_list = [targets[b]["polygons"][t] for t in tgt_idx]

            for pred_poly, gt_poly in zip(pred_list, gt_list):
                t_gt = self._compute_arc_length_t(gt_poly)
                phi_gt = self._phases_from_t(t_gt)

                n = pred_poly.shape[0]
                pred_phi = torch.linspace(
                    0, 2 * math.pi * (1 - 1/n),
                    n, device=pred_poly.device
                )

                snapped = self._snap(pred_phi, phi_gt, n)

                t_pred = self._compute_arc_length_t(pred_poly)

                # L1
                p_pred_L1 = self._interp(pred_poly, t_pred, snapped)
                p_gt_L1   = self._interp(gt_poly,  t_gt,   snapped)
                L1 = F.smooth_l1_loss(p_pred_L1, p_gt_L1, reduction='mean')

                # L2
                p_pred_L2 = self._interp(pred_poly, t_pred, phi_gt)
                p_gt_L2   = self._interp(gt_poly,  t_gt,   phi_gt)
                L2 = F.smooth_l1_loss(p_pred_L2, p_gt_L2, reduction='mean')

                all_losses.append(0.5 * (L1 + L2))

        if len(all_losses) == 0:
            return {"loss_polygon": torch.tensor(0.0, device=pred_polys.device)}

        loss = torch.stack(all_losses).sum() / max(num_boxes, 1)
        return {"loss_polygon": loss}

    # ---------- EFD L1 ----------
    def loss_efd(self, outputs, targets, indices, num_boxes, log=True):
        src_efd = outputs['pred_efd']
        idx = self._get_src_permutation_idx(indices)
        pred = src_efd[idx]

        target = torch.cat(
            [t["efd_params"][J] for t, (_, J) in zip(targets, indices)],
            dim=0
        )

        loss = F.l1_loss(pred, target, reduction='none').sum() / max(num_boxes, 1)
        return {"loss_efd": loss}

    # ============================================
    # get_loss — only the 4 used losses
    # ============================================
    def get_loss(self, name, outputs, targets, indices, num_boxes, **kwargs):
        loss_map = {
            "vfl": self.loss_vfl,
            "boxes": self.loss_boxes,
            "efd": self.loss_efd,
            "polygon": self.loss_polygon,
        }
        return loss_map[name](outputs, targets, indices, num_boxes, **kwargs)

    # ============================================
    # forward — standard DETR Criterion forward
    # ============================================
    def forward(self, outputs, targets):
        """
        outputs: dict of:
            pred_logits (B,Q,C)
            pred_boxes  (B,Q,4)
            pred_efd    (B,Q,D)
            pred_polygons (B,Q,P,2)
        targets: list of dicts
        """
        # 1) Hungarian matching
        indices = self.matcher(outputs, targets)

        # number of GTs
        num_boxes = sum(len(t["labels"]) for t in targets)
        num_boxes = max(num_boxes, 1)

        # 2) compute each loss
        loss_dict = {}
        for loss_name in self.losses:
            loss_dict.update(
                self.get_loss(loss_name, outputs, targets, indices, num_boxes)
            )

        # 3) weight losses
        final = {}
        for k, v in loss_dict.items():
            if k in self.weight_dict:
                final[k] = v * self.weight_dict[k]

        return final
class SetCriterion(nn.Module):
    """
    EFDTR용 통합 loss 모듈

    losses:
      - 'vfl'     : Varifocal loss (classification, IoU soft label)
      - 'boxes'   : L1 + GIoU bbox regression
      - 'efd'     : EFD coefficient L1
      - 'polygon' : cyclic-shift 정렬 Smooth-L1 polygon loss
    """

    def __init__(
        self,
        matcher,                # HungarianMatcher
        weight_dict,            # {"loss_vfl":1, "loss_bbox":5, ...}
        losses,
        alpha=0.75,             # VFL alpha
        gamma=2.0,              # VFL gamma
        eos_coef=1e-4,
        num_classes=1,
    ):
        super().__init__()
        self.matcher = matcher
        self.weight_dict = weight_dict
        self.losses = losses
        self.alpha = alpha
        self.gamma = gamma
        self.num_classes = num_classes

        # background class weight (거의 안 쓰지만 일단 유지)
        empty_weight = torch.ones(num_classes + 1)
        empty_weight[-1] = eos_coef
        self.register_buffer("empty_weight", empty_weight)

    # ------------------------------------------------
    # permutation index helpers (DETR 스타일)
    # ------------------------------------------------
    @staticmethod
    def _get_src_permutation_idx(indices):
        # indices: list[(src_idx, tgt_idx)] (batch 길이)
        batch_idx = torch.cat(
            [torch.full_like(src, i) for i, (src, _) in enumerate(indices)]
        )
        src_idx = torch.cat([src for (src, _) in indices])
        return batch_idx, src_idx

    @staticmethod
    def _get_tgt_permutation_idx(indices):
        batch_idx = torch.cat(
            [torch.full_like(tgt, i) for i, (_, tgt) in enumerate(indices)]
        )
        tgt_idx = torch.cat([tgt for (_, tgt) in indices])
        return batch_idx, tgt_idx

    # ------------------------------------------------
    # 1) Varifocal classification loss
    #     - soft label = IoU( pred_box, gt_box )
    # ------------------------------------------------
    def loss_vfl(self, outputs, targets, indices, num_boxes, log=True):
        """
        outputs: dict with key 'pred_logits', 'pred_boxes'
          pred_logits: (B, Q, 1)  -- 건물 vs background 이진
        """
        assert 'pred_logits' in outputs and 'pred_boxes' in outputs

        src_logits = outputs['pred_logits']      # (B, Q, 1)
        src_boxes  = outputs['pred_boxes']       # (B, Q, 4)

        bs, num_queries, _ = src_logits.shape

        # 매칭된 쿼리 인덱스
        idx = self._get_src_permutation_idx(indices)
        matched_pred_boxes = src_boxes[idx]      # (N_pos, 4)

        # 대응 GT box (cxcywh)
        target_boxes = torch.cat(
            [t["boxes"][J] for t, (_, J) in zip(targets, indices)], dim=0
        )

        # IoU (axis-aligned) → soft target score
        iou, _ = box_iou(
            box_cxcywh_to_xyxy(matched_pred_boxes),
            box_cxcywh_to_xyxy(target_boxes),
        )
        iou_diag = torch.diag(iou).detach()      # (N_pos,)

        # GT class: 0..num_classes-1, background = num_classes
        target_classes_o = torch.cat(
            [t["labels"][J] for t, (_, J) in zip(targets, indices)]
        )
        target_classes = torch.full(
            src_logits.shape[:2],
            self.num_classes,
            dtype=torch.int64,
            device=src_logits.device,
        )   # (B,Q)
        target_classes[idx] = target_classes_o

        # one-hot, background 제거
        target_onehot = F.one_hot(
            target_classes, num_classes=self.num_classes + 1
        )[..., :-1]    # (B,Q,C)

        # score map: positive 위치에 IoU, 나머지 0
        score_map = torch.zeros_like(target_classes, dtype=src_logits.dtype)
        score_map[idx] = iou_diag
        score_map = score_map.unsqueeze(-1) * target_onehot  # (B,Q,C)

        # Varifocal weight
        pred_sigmoid = src_logits.sigmoid().detach()
        weight = self.alpha * pred_sigmoid.pow(self.gamma) * (1 - target_onehot) \
                 + score_map

        # BCE with logits, class-wise
        loss = F.binary_cross_entropy_with_logits(
            src_logits, score_map, weight=weight, reduction='none'
        )  # (B,Q,C)
        # query 차원을 평균, 전체 sum 후 박스 개수로 정규화
        loss = loss.mean(-1).sum() / max(num_boxes, 1)

        return {"loss_vfl": loss}

    # ------------------------------------------------
    # 2) Box regression: L1 + GIoU
    # ------------------------------------------------
    def loss_boxes(self, outputs, targets, indices, num_boxes, log=True):
        assert 'pred_boxes' in outputs

        idx = self._get_src_permutation_idx(indices)
        src_boxes = outputs['pred_boxes'][idx]

        target_boxes = torch.cat(
            [t["boxes"][J] for t, (_, J) in zip(targets, indices)], dim=0
        )

        # L1
        loss_l1 = F.l1_loss(src_boxes, target_boxes, reduction='none').sum()

        # GIoU
        giou = generalized_box_iou(
            box_cxcywh_to_xyxy(src_boxes),
            box_cxcywh_to_xyxy(target_boxes),
        )
        loss_giou = (1. - torch.diag(giou)).sum()

        return {
            "loss_bbox": loss_l1 / max(num_boxes, 1),
            "loss_giou": loss_giou / max(num_boxes, 1),
        }

    # ------------------------------------------------
    # 3) EFD coefficient L1
    # ------------------------------------------------
    def loss_efd(self, outputs, targets, indices, num_boxes, log=True):
        if 'pred_efd' not in outputs:
            return {"loss_efd": torch.tensor(0.0, device=outputs['pred_boxes'].device)}

        src_efd = outputs['pred_efd']   # (B,Q,L)
        idx = self._get_src_permutation_idx(indices)
        pred = src_efd[idx]

        target = torch.cat(
            [t["efd_params"][J] for t, (_, J) in zip(targets, indices)],
            dim=0
        )

        loss = F.l1_loss(pred, target, reduction='none').sum() / max(num_boxes, 1)
        return {"loss_efd": loss}

    # ------------------------------------------------
    # Polygon helper: arc-length, phase, interpolate
    # ------------------------------------------------
    @staticmethod
    def _compute_arc_length_t(poly: torch.Tensor) -> torch.Tensor:
        """
        poly: (P,2)   닫힌 폴리곤 (마지막점!=첫점이라 가정, 내부에서 자동 loop)
        return: (P,) 누적 arc length (eq.3)
        """
        P = poly.shape[0]
        diffs = poly[(torch.arange(P) + 1) % P] - poly
        seg_len = torch.norm(diffs, dim=-1)      # (P,)
        t = torch.cumsum(seg_len, dim=0)         # (P,)
        return t

    @staticmethod
    def _phases_from_t(t: torch.Tensor) -> torch.Tensor:
        T = t[-1]
        return 2 * math.pi * t / T

    @staticmethod
    def _interp(poly: torch.Tensor, t: torch.Tensor, phases: torch.Tensor) -> torch.Tensor:
        """
        poly : (P,2)
        t    : (P,) arc-length cumulative
        phases: (K,) 원하는 phase들 (0~2π)
        → poly를 arc-length 기준 선형보간해서 (K,2) 반환
        """
        T = t[-1]
        t_phase = phases * T / (2 * math.pi)     # (K,)

        # 각 t_phase가 속하는 segment index 찾기
        idx = torch.searchsorted(t, t_phase, right=True) - 1
        idx = idx.clamp(0, poly.size(0) - 1)

        t0 = t[idx]
        t1 = t[(idx + 1) % poly.size(0)]
        w = ((t_phase - t0) / (t1 - t0).clamp(min=1e-6)).unsqueeze(-1)

        p0 = poly[idx]
        p1 = poly[(idx + 1) % poly.size(0)]
        return (1. - w) * p0 + w * p1

    # ------------------------------------------------
    # 4) Polygon loss (간단한 phase snapping 구현)
    # ------------------------------------------------
    def loss_polygon(self, outputs, targets, indices, num_boxes, log=True):
        if 'pred_polygons' not in outputs:
            return {"loss_polygon": torch.tensor(0.0, device=outputs['pred_boxes'].device)}

        pred_polys = outputs['pred_polygons']    # (B, Q, P, 2)
        all_losses = []

        for b, (src_idx, tgt_idx) in enumerate(indices):
            if len(src_idx) == 0:
                continue

            # 매칭된 쿼리/GT 쌍들
            pred_list = [pred_polys[b, s] for s in src_idx]
            gt_list   = [targets[b]["polygons"][t] for t in tgt_idx]

            for pred_poly, gt_poly in zip(pred_list, gt_list):
                # pred: (P,2)  — EFD inverse로 나온 128 포인트
                P = pred_poly.shape[0]

                # GT 폴리곤을 pred와 같은 포인트 개수로 재샘플링
                gt_poly = gt_poly.to(pred_poly.device)
                t_gt = self._compute_arc_length_t(gt_poly)
                phi_uniform = torch.linspace(
                    0, 2 * math.pi * (1 - 1.0 / P),
                    P, device=pred_poly.device
                )
                gt_resampled = self._interp(gt_poly, t_gt, phi_uniform)  # (P,2)

                # ---- L1: cyclic shift 정렬 (phase snapping 근사) ----
                # 모든 shift 에 대해 pred vs rolled(gt) SmoothL1 계산 후 최소 선택
                losses_shift = []
                for shift in range(P):
                    gt_shift = gt_resampled.roll(-shift, dims=0)
                    loss_shift = F.smooth_l1_loss(
                        pred_poly, gt_shift, reduction='mean'
                    )
                    losses_shift.append(loss_shift)
                L1 = torch.stack(losses_shift).min()

                # ---- L2: 정렬 없이 직접 정합 (global constraint) ----
                L2 = F.smooth_l1_loss(pred_poly, gt_resampled, reduction='mean')

                all_losses.append(0.5 * (L1 + L2))

        if len(all_losses) == 0:
            # 이 batch에 positive가 하나도 없을 때
            device = outputs['pred_boxes'].device
            return {"loss_polygon": torch.tensor(0.0, device=device)}

        loss = torch.stack(all_losses).sum() / max(num_boxes, 1)
        return {"loss_polygon": loss}

    # ------------------------------------------------
    # loss dispatcher
    # ------------------------------------------------
    def get_loss(self, name, outputs, targets, indices, num_boxes, **kwargs):
        loss_map = {
            "vfl": self.loss_vfl,
            "boxes": self.loss_boxes,
            "efd": self.loss_efd,
            "polygon": self.loss_polygon,
        }
        return loss_map[name](outputs, targets, indices, num_boxes, **kwargs)

    # ------------------------------------------------
    # forward
    # ------------------------------------------------
    def forward(self, outputs, targets):
        """
        outputs:
          - decoder 그대로 쓴 경우:
              {"class_logits","boxes","efd_params","polygons"}  (EFDTR decoder out)
          - 이미 매핑한 경우:
              {"pred_logits","pred_boxes","pred_efd","pred_polygons"}

        targets: list[dict], 각 dict 안에
          "labels": (Ni,), "boxes": (Ni,4), "polygons": (Ni,P_i,2), "efd_params": (Ni,L)
        """

        # EFDTR decoder raw 출력이면, DETR 스타일 키로 매핑
        if "pred_logits" not in outputs and "class_logits" in outputs:
            outputs = {
                **outputs,
                "pred_logits":  outputs["class_logits"],
                "pred_boxes":   outputs["boxes"],
                "pred_efd":     outputs.get("efd_params"),
                "pred_polygons": outputs.get("polygons"),
            }

        # 1) Hungarian matching
        indices = self.matcher(outputs, targets)

        # 2) number of GT boxes (정규화용)
        num_boxes = sum(len(t["labels"]) for t in targets)
        num_boxes = max(num_boxes, 1)

        # 3) loss 계산
        loss_dict = {}
        for loss_name in self.losses:
            loss_dict.update(
                self.get_loss(loss_name, outputs, targets, indices, num_boxes)
            )

        # 4) weight 적용
        final = {}
        for k, v in loss_dict.items():
            if k in self.weight_dict:
                final[k] = v * self.weight_dict[k]

        return final

In [ ]:
# ====== SMOKE TEST FOR EFDTR ======
import torch
import torch.nn as nn

# 만약 다른 파일에 있다면 from XXX import EFDTRModel, HungarianMatcher, SetCriterion
# 여기서는 현재 노트북에 이미 정의됐다고 가정

def build_model_matcher_criterion(
    device="cuda" if torch.cuda.is_available() else "cpu"
):
    # 1) 모델
    model = EFDTRModel(
        backbone_name="swin_base_patch4_window7_224",  # 네가 쓰는 이름으로 맞춰도 됨
        num_classes=2,        # (배경+건물) 셋팅이면 2, 순수 건물만 1로 바꿔도 됨
        num_queries=100,
        num_denoising=20,
        num_harmonics=1,
        num_points_poly=128,
    ).to(device)

    # 2) 매쳐
    matcher = HungarianMatcher(
        weight_dict={
            "cost_class": 1.0,
            "cost_bbox": 5.0,
            "cost_giou": 2.0,
        },
        use_focal_loss=True,
    )

    # 3) 크리테리온 (SetCriterion는 내가 방금 짜준 버전으로 교체해둔 상태라고 가정)
    weight_dict = {
        "loss_vfl": 1.0,
        "loss_bbox": 5.0,
        "loss_giou": 2.0,
        "loss_efd": 1.0,
        "loss_polygon": 2.0,
    }
    criterion = SetCriterion(
        matcher=matcher,
        weight_dict=weight_dict,
        losses=["vfl", "boxes", "efd", "polygon"],
        num_classes=1,   # 실제 클래스 개수 (건물 1개라고 가정)
    ).to(device)

    return model, matcher, criterion


def make_dummy_targets(
    batch_size,
    num_gt_per_image=3,
    num_harmonics=1,
    num_points_poly=128,
    device="cuda" if torch.cuda.is_available() else "cpu",
):
    targets = []
    efd_dim = 4 * num_harmonics + 1  # 4n+1

    for _ in range(batch_size):
        N = num_gt_per_image

        # GT boxes: cx,cy,w,h in [0,1]
        boxes = torch.rand(N, 4, device=device)
        boxes[:, 2:] *= 0.5  # w,h 를 너무 크게 만들지 않게

        # GT labels: 전부 class 0 (건물)
        labels = torch.zeros(N, dtype=torch.long, device=device)

        # GT EFD params (phase 정규화 전용 dummy)
        efd_params = torch.rand(N, efd_dim, device=device)

        # GT polygons: (N, 128, 2), 좌표 [0,1]
        polygons = torch.rand(N, num_points_poly, 2, device=device)

        targets.append(
            {
                "boxes": boxes,
                "labels": labels,
                "efd_params": efd_params,
                "polygons": polygons,
            }
        )
    return targets


def smoke_train_step_EFDTR():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"[SMOKE] Device: {device}")

    # 1) 모델/매쳐/크리테리온 생성
    model, matcher, criterion = build_model_matcher_criterion(device)
    model.train()

    # 2) 더미 입력 / 타깃 생성
    batch_size = 2
    H = W = 224  # swin_base_patch4_window7_224 에 맞춰 224 사용
    images = torch.randn(batch_size, 3, H, W, device=device)

    targets = make_dummy_targets(
        batch_size=batch_size,
        num_gt_per_image=3,
        num_harmonics=1,
        num_points_poly=128,
        device=device,
    )

    # 3) Forward
    outputs = model(images, targets)  # training mode → (out_main, out_dn, dn_meta) 또는 dict 하나

    if isinstance(outputs, tuple):
        out_main, out_dn, dn_meta = outputs
        print("[Decoder Output] (training with DN)")
        print(" main class_logits:", out_main["class_logits"].shape)
        print(" main boxes      :", out_main["boxes"].shape)
        print(" main efd_params :", out_main["efd_params"].shape)
        print(" main polygons   :", out_main["polygons"].shape)

        print(" dn class_logits :", out_dn["class_logits"].shape)
        print(" dn boxes        :", out_dn["boxes"].shape)
        print(" dn efd_params   :", out_dn["efd_params"].shape)
        print(" dn polygons     :", out_dn["polygons"].shape)

        outputs_for_loss = out_main  # 보통 메인 쿼리에만 loss 걸기
    else:
        outputs_for_loss = outputs
        print("[Decoder Output] (no DN)")
        print(" class_logits:", outputs_for_loss["class_logits"].shape)
        print(" boxes      :", outputs_for_loss["boxes"].shape)
        print(" efd_params :", outputs_for_loss["efd_params"].shape)
        print(" polygons   :", outputs_for_loss["polygons"].shape)

    # 4) Loss 계산
    loss_dict = criterion(outputs_for_loss, targets)
    total_loss = sum(loss_dict.values())

    print("\n[Loss components]")
    for k, v in loss_dict.items():
        print(f" {k}: {float(v):.4f}")
    print(f" total_loss: {float(total_loss):.4f}")

    # 5) Backward & optimizer.step() 테스트
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    optimizer.zero_grad(set_to_none=True)
    total_loss.backward()
    optimizer.step()

    print("\n[SMOKE] ✔ Backward & optimizer.step() success!")


# 실제로 한 번 호출
smoke_train_step_EFDTR()


[SMOKE] Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

/tmp/ipython-input-2143581798.py:585: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  offset_normalizer = torch.tensor(value_spatial_shapes)


[Decoder Output] (training with DN)
 main class_logits: torch.Size([2, 100, 1])
 main boxes      : torch.Size([2, 100, 4])
 main efd_params : torch.Size([2, 100, 5])
 main polygons   : torch.Size([2, 100, 128, 2])
 dn class_logits : torch.Size([2, 36, 1])
 dn boxes        : torch.Size([2, 36, 4])
 dn efd_params   : torch.Size([2, 36, 5])
 dn polygons     : torch.Size([2, 36, 128, 2])

[Loss components]
 loss_vfl: 19.9074
 loss_bbox: 1.9924
 loss_giou: 1.8952
 loss_efd: 2.3718
 loss_polygon: 0.7861
 total_loss: 26.9528


/tmp/ipython-input-114033574.py:138: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  print(f" {k}: {float(v):.4f}")



[SMOKE] ✔ Backward & optimizer.step() success!
